In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
from tkinter import Tk
from tkinter import filedialog
from tkinter import simpledialog

In [ ]:
L_paper_white = 94.61
L_darkest = 1.85

In [ ]:
root = Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
txt_files = filedialog.askopenfilenames(title="Datacolor Measurement .txt file(s)", filetypes=[("TXT files", "*.txt")])

In [ ]:
dfs = []
for txt in txt_files:
    df = pd.read_csv(txt, header=None, usecols=[0], sep='\t')
    dfs.append(df)
# rename the columns in each dataframe
for i in range(len(dfs)):
    dfs[i].rename(columns={0:"Text_File_{}".format(i+1)}, inplace=True)
# make the value steps
value_steps = []
num_steps = len(dfs[0].index)
step_increment = 1.0 / (num_steps - 1)
min_val = 0
for i in range(num_steps):
    value_steps.append(round(min_val + (i * step_increment), 4) * 100)
# check to see if the value steps' order matches the first dataframe
mindf = dfs[0][['Text_File_1']].idxmin().values[0]
maxdf = dfs[0][['Text_File_1']].idxmax().values[0]
# create a dataframe using the value steps as the index
datacolor_df = pd.DataFrame({"Linear":value_steps}, index=value_steps)
# add the values from the TXT dataframes
column_names = []
for i in range(len(dfs)):
    col_vals = dfs[i]['Text_File_{}'.format(i+1)].tolist()
    # make sure the list values are increasing, not decreasing
    if col_vals[0] > col_vals[-1]:
        col_vals.reverse()
    datacolor_df['Text_File_{}'.format(i+1)] = col_vals
    column_names.append('Text_File_{}'.format(i+1))
# average the text file columns
datacolor_df["Avg_TXT_values"] = datacolor_df[column_names].mean(axis=1)

In [ ]:
def interpolate(xval, df, xcol, ycol):
    """
    This function returns an interpolated value from a dataframe at a specified X value.
    
    Inputs
    ------
    xval: numeric - the X value at which you wish to get the interpolated Y value
    df: pandas dataframe - the dataframe containing the data to be interpolated
    xcol: string - the name of the column in the dataframe containing X values
    ycol: string - the name of the column in the dataframe containing Y values
    
    Returns
    -------
    (interpolated_value, X, Y)
    """
    return np.interp([xval], df[xcol], df[ycol])

In [ ]:
# scale the values to fit within the minimum and maximum paper values
min_val = datacolor_df['Avg_TXT_values'].min()
if min_val > L_darkest:
    min_val = L_darkest
datacolor_df['Avg_TXT_values'] = (datacolor_df['Avg_TXT_values'] - min_val)
scaling_fact = L_paper_white / datacolor_df['Avg_TXT_values'].max()
datacolor_df['Avg_TXT_values'] = (datacolor_df['Avg_TXT_values'] * scaling_fact)

In [ ]:
# smooth the averaged values
smoothing_factor_value = simpledialog.askinteger(title="Smoothing Factor", prompt="Enter a smoothing factor.\nBigger numbers mean a smoother result.\nValues between 100 and 500 are most common.", initialvalue=110)
if smoothing_factor_value == None:
    smoothing_factor_value = 110
avg_spline = scipy.interpolate.UnivariateSpline(datacolor_df['Linear'], datacolor_df['Avg_TXT_values'], k=5)
avg_spline.set_smoothing_factor(smoothing_factor_value)
# get the smoothed readings, and make sure they're strictly increasing
smoothed_vals = avg_spline(datacolor_df['Linear']).tolist()
output_vals = []
previous_val = -1
for val in smoothed_vals:
    if val > previous_val:
        output_vals.append(val)
        previous_val = val
    else:
        output_vals.append(previous_val)
# add the values to the dataframe
datacolor_df['Smoothed_TXT'] = output_vals

In [ ]:
# flip the values around the linear line - this requires interpolation
new_curve_vals = []
for i in value_steps:
    ival = interpolate(i, datacolor_df, 'Smoothed_TXT', 'Linear')
    new_curve_vals.append(ival[0])
datacolor_df['Curve'] = new_curve_vals
datacolor_df[['Linear','Avg_TXT_values','Smoothed_TXT','Curve']].plot(figsize=(7,7), grid=True)
plt.show()

In [ ]:
# scale the values to fit within 0.0 to 100.0
min_val = datacolor_df['Curve'].min()
datacolor_df['Scaled_Curve'] = (datacolor_df['Curve'] - min_val)
max_val = datacolor_df['Scaled_Curve'].max()
scaling_fact = 100.0 / max_val
datacolor_df['Scaled_Curve'] = datacolor_df['Scaled_Curve'] * scaling_fact
datacolor_df[['Linear','Avg_TXT_values','Curve','Scaled_Curve']].plot(figsize=(7,7), grid=True)
plt.show()

In [ ]:
# convert the curve so it falls within 0.0 and 1.0
converted_vals = [e/100.0 for e in datacolor_df['Scaled_Curve'].tolist()]

# get the output filename
cube_filename = simpledialog.askstring("Output .cube File Name", "Enter the name for the output .cube file (no .cube extension)")
cube_filename = cube_filename.replace(" ", "_")

# save the new curve to a .cube file
out_cube = os.path.join(os.path.dirname(txt_files[0]), "{}.cube".format(cube_filename))
with open(out_cube, 'w') as cube:
    cube.write("#Created by: Datacolor_Generate_Curve\nTITLE  \"{}\"\nLUT_1D_SIZE {}\nDOMAIN_MIN 0.0 0.0 0.0\nDOMAIN_MAX 1.0 1.0 1.0\n#LUT data points\n".format(cube_filename, len(converted_vals)))
    line_template = "{:<08} {:<08} {:<08}\n"
    for row in converted_vals:
        if row < 0.0:
            val = 0.0
        elif row > 1.0:
            val = 1.0
        else:
            val = round(row, 6)
        cube.write(line_template.format(val, val, val))
    cube.write("#END data\n")
print("Saved {}".format(out_cube))